In [2]:
keyword = "Artificial Intelligence"
article_count = 30
topic_count = 10
# max_period_hours = 3
news_country = "United States"

In [3]:
from dotenv import load_dotenv
from autogen import UserProxyAgent, AssistantAgent
from twikit import Client
import os

# Load environment variables
load_dotenv()
GROQ_MODEL_NAME = os.getenv("GROQ_MODEL_NAME")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_BASE = os.getenv("GROQ_API_BASE")
RELEASE = os.getenv("RELEASE")

if(RELEASE == "PROD"):
    USERNAME = os.getenv("XUSERNAME")
    EMAIL = os.getenv("XEMAIL")
    PASSWORD = os.getenv("XPASSWORD")
else:
    USERNAME = os.getenv("XUSERNAME_TEST")
    EMAIL = os.getenv("XEMAIL_TEST")
    PASSWORD = os.getenv("XPASSWORD_TEST")

# Config dictionary
llm_config = {
    "cache_seed": 42,
    "config_list": [{
        "model": GROQ_MODEL_NAME,
        "api_key": GROQ_API_KEY,
        "base_url": GROQ_API_BASE
    }],
}

# # Initialize client
if RELEASE != 'DEV' and 'x_client' not in globals():
    x_client = Client('en-US')

    x_client.login(
        auth_info_1=USERNAME,
        auth_info_2=EMAIL,
        password=PASSWORD
    )
    print("Client initialized")


Client initialized


In [4]:
import random
import pandas as pd
import os
from typing import Annotated

# topics_file = '.cache/topics.csv'

# #reset the topics
# if os.path.isfile(topics_file):
#     os.remove(topics_file)

# def topic_selection_tool(topics_list: Annotated[list, "The list of topics"] = None) -> str:
#     #set df_topics topic column from the topics_list and set status to pending if it's not already in the df_topics
#     if os.path.isfile(topics_file):
#         df_topics = pd.read_csv(topics_file, index_col='Unnamed: 0')
#     else:
#         df_topics = pd.DataFrame(columns=['topic', 'status'])  # Define the variable with a default value
#     if topics_list:
#         for topic in topics_list:
#             if topic not in df_topics['topic'].values:
#                 df_topics = pd.concat([df_topics, pd.DataFrame([[topic, 'pending']], columns=df_topics.columns)], ignore_index=True)
    
#     topics_list = df_topics[df_topics['status'] == 'pending']['topic'].values.tolist()
#     if(len(topics_list) == 0):
#         return "No more topics to select"
    
#     topic_selected = random.choice(topics_list)
#     df_topics.loc[df_topics['topic'] == topic_selected, 'status'] = 'selected'
    
#     df_topics.to_csv(topics_file)
#     return topic_selected


In [5]:
from gnews import GNews
from numpy import floor
from pyshorteners import Shortener
from newspaper import Article

urls_file = '.cache/urls.csv'

# def select_random_article(news_list):
#     if not os.path.isfile(urls_file):
#         df_urls = pd.DataFrame(columns=['urls', 'status'])  # Define the variable with a default value
#         df_urls.to_csv(urls_file)
#     else:
#         df_urls = pd.read_csv(urls_file, index_col='Unnamed: 0')
    
#     news, article = None, None
#     while True:
#         print(f'NEWS LIST: {news_list}')
#         #remove any news from the news_list if it is already in the csv file
#         news_list = [news for news in news_list if news['url'] not in df_urls['urls'].values]
#         print(f'FILTERED NEWS LIST: {news_list}')
#         if not news_list:
#             print("No more news to select")
#             return None, None
#         # news = random.choice(news_list)
#         news = random.choice(news_list)
#         try:
#             article = Article(news['url'])
#             article.download()
#             article.parse()

#             if article.text and len(article.text.strip().split('\n')) > 1:
#                 # Append the URL and status to the DataFrame
#                 df_urls = pd.concat([pd.DataFrame([[news['url'], 'success']], columns=df_urls.columns), df_urls], ignore_index=True)
#                 df_urls.to_csv(urls_file)
#                 break
#             else:
#                 df_urls = pd.concat([pd.DataFrame([[news['url'], 'empty']], columns=df_urls.columns), df_urls], ignore_index=True)
#                 df_urls.to_csv(urls_file)
#                 continue
#         except Exception as e:
#             # Append the URL and status to the DataFrame
#             df_urls = pd.concat([pd.DataFrame([[news['url'], 'error']], columns=df_urls.columns), df_urls], ignore_index=True)
#             df_urls.to_csv(urls_file)
#             print(f"Error selecting article: {str(e)}")
#             continue
#     return news, article


In [6]:
def select_news_article_tool(news_list):
    if not os.path.isfile(urls_file):
        df_urls = pd.DataFrame(columns=['urls', 'status'])  # Define the variable with a default value
        df_urls.to_csv(urls_file)
    else:
        df_urls = pd.read_csv(urls_file, index_col='Unnamed: 0')
    
    news_list = [news for news in news_list if news['url'] not in df_urls['urls'].values]
    
    #deduplicate the news_list based on urls
    def deduplicate_news_list(news_list):
        seen_urls = set()
        unique_news_list = []
        for news_item in news_list:
            url = news_item.get('url')
            if url not in seen_urls:
                seen_urls.add(url)
                unique_news_list.append(news_item)
        return unique_news_list
    news_list = deduplicate_news_list(news_list)

    if len(news_list) == 0:
        return [], []

    article_list = []
    final_news_list = []
    for news in news_list:
        try:
            article = Article(news['url'])
            article.download()
            article.parse()

            if article.text and len(article.text.strip().split('\n')) > 1:
                # Append the URL and status to the DataFrame
                df_urls = pd.concat([pd.DataFrame([[news['url'], 'success']], columns=df_urls.columns), df_urls], ignore_index=True)
                df_urls.to_csv(urls_file)
                final_news_list.append(news)
                article_list.append(article)
                continue
            else:
                df_urls = pd.concat([pd.DataFrame([[news['url'], 'empty']], columns=df_urls.columns), df_urls], ignore_index=True)
                df_urls.to_csv(urls_file)
                continue
        except Exception as e:
            df_urls = pd.concat([pd.DataFrame([[news['url'], 'error']], columns=df_urls.columns), df_urls], ignore_index=True)
            df_urls.to_csv(urls_file)
            print(f"Error selecting article: {str(e)}")
            continue

    return final_news_list, article_list

def get_news_articles_tool(topics_list: Annotated[list, "The list of topics"], count: Annotated[int, "The number of news articles to collect from the internet"]) -> str:
    google_news = GNews()
    google_news.max_results = int(floor(count/len(topics_list))) # number of responses for one topic
    google_news.language = 'english'  # News in a specific language
    google_news.country = news_country  # News from a specific country
    google_news.period = '1h'  # Adjust period in hours

    s = Shortener(timeout=5)
    
    raw_news_list = []
    for topic in topics_list:
        print(f"FETCHING NEWS ON TOPIC: {topic}")
        raw_news_list.extend(google_news.get_news(topic))
        
    if len(raw_news_list) == 0:
        return None
    
    news_list, article_list = select_news_article_tool(raw_news_list)
    print(f"NEWS LIST len: {len(news_list)}")
    print(f"ARTICLE LIST len: {len(article_list)}")

    result = ''
    if len(news_list) > 0 and len(article_list) > 0:
        # get the news['url'] for each news
        news_url_list = [news['url'] for news in news_list]
        short_urls = [s.tinyurl.short(url) for url in news_url_list]
        article_text_list = [article.text for article in article_list]

        for i in range(len(news_list)):
            # do this until the result length is less than 31000
            if len(result) > 30000:
                break
            result += ("""NEWS {n} TITLE: {title}
NEWS {n} CONTENT: {content}
NEWS {n} SOURCE: {url}

"""
        ).format(
            n = i+1,
            title=news_list[i]['title'],
            content=article_text_list[i].replace('\n\n', '\n')[:1000],
            url=short_urls[i]
        )

    return result[:4500]


In [7]:
# Test the function
# topics_list = ['Artificial Intelligence', 'Machine Learning', 'Deep Learning', 'Data Science', 'Computer Vision']
# news_count = 10
# res = get_news_articles_tool(topics_list, news_count)

In [8]:
import re
from datetime import datetime
import pytz

def merge_tweets(tweet_list: Annotated[list, "The list of tweets to merge"]) -> None:
    merged_tweets = []
    current_tweet = ""

    for tweet in tweet_list:
        if len(current_tweet) + len(tweet) <= 278:
            current_tweet += f'{tweet}\n\n'
        else:
            merged_tweets.append(current_tweet)
            current_tweet = tweet
    if current_tweet:
        merged_tweets.append(current_tweet)
    
    return merged_tweets

def add_source_urls(tweet_list: Annotated[list, "The list of tweets to post"], source_list: Annotated[list, "The list of 'https://tinyurl.com/' source URLs for each tweet"]) -> list:
    if len(tweet_list) == len(source_list):
        del_index = []
        for i in range(len(source_list)):
            if not re.search(r'https://tinyurl\.com/[a-zA-Z0-9]{8}', tweet_list[i]): # if the tweet does not contain a source
                if re.search(r'https://tinyurl\.com/[a-zA-Z0-9]{8}', source_list[i]): # if we have a source seperately
                    tweet_list[i] = tweet_list[i][:278-len(source_list[i])]
                    tweet_list[i] += f"\n{source_list[i]}"
    # uncomment below five lines to don't post a news if it doesn't have a source
        #         else:
        #             del_index.append(i)
        # for i in sorted(del_index, reverse=True):
        #     del tweet_list[i]
        #     del source_list[i]
        
    return tweet_list

def get_intro_tweet() -> str:
    now = datetime.now(pytz.utc)
    eastern = pytz.timezone('America/New_York')
    now_eastern = now.astimezone(eastern)

    last_hour = now_eastern.replace(minute=0, second=0, microsecond=0)

    formatted_datetime = last_hour.strftime("%I:00%p EST, %B %d, %Y")

    return f"""These are the AI news within the last 1 hour:
{formatted_datetime}"""


In [9]:
import time


def write_tweet_tool(tweet_list: Annotated[list, "The list of tweets to post"], source_list: Annotated[list, "The list of 'https://tinyurl.com/' source URL for each tweet"]) -> str:
    # tweet_list = [get_intro_tweet()] + merge_tweets(add_source_urls(tweet_list, source_list))
    tweet_list = [get_intro_tweet()] + add_source_urls(tweet_list, source_list)

    posts = ''
    # for tweet, source in zip(tweet_list, source_list):
    for i in range(len(tweet_list)):
    #     if 'https://tinyurl.com/' in tweet_list[i]:
    #         if len(tweet_list[i]) > 280:
    #             tweet_list[i] = tweet_list[i][:276-len(source)] + '...' + f"\n{source}"
    #     else:
    #         if len(tweet_list[i]) <= 279-len(source):
    #             tweet_list[i] += f"\n{source}"
    #         else:
    #             tweet_list[i] = tweet_list[i][:276-len(source)] + '...' + f"\n{source}"
        try:
            # final tweet length check for redundancy 
            if len(tweet_list[i]) > 280:
                tweet_list[i] = tweet_list[i][:276] + '...'

            if RELEASE != "DEV":
                if i==0:
                    last_tweet = x_client.create_tweet(
                        text=tweet_list[i],
                    )
                else:
                    last_tweet = x_client.create_tweet(
                        text=tweet_list[i],
                        reply_to=last_tweet.id
                    )
            
            posts += ("""
Tweet: {tweet}
Length: {length}
                      
                      """).format(tweet=tweet_list[i], length=len(tweet_list[i]))
            
            time.sleep(1)
        except Exception as e:
            error_message = f"Failed to post tweet: {str(e)}"
            print(error_message)
            continue

    if(posts == ''):
        posts = "No tweets posted"
        
    return posts


In [10]:
# topic_selector_agent = AssistantAgent(
#     "topic_selector_agent",
#     llm_config=llm_config,
#     system_message=f"You are good at writing a list of closely related topics to the given topic(including the given topic) and then choose any one out of them. Use the provided tools for both topic selection and to collect new articles.",
#     max_consecutive_auto_reply=1
# )

news_collector_agent = AssistantAgent(
    "news_collector_agent",
    llm_config=llm_config,
    system_message=f"""You are good at collecting recent news articles about a given keyword on the internet. 
    You should generate a list of {topic_count} topics closely related to the given keyword. 
    Use the provided tool to collect news about the generated list of topics.""",
    max_consecutive_auto_reply=1
)

tweet_writer_agent = AssistantAgent(
    "tweet_writer_agent",
    llm_config=llm_config,
    system_message=f"""You are an autonomous twitter bot that's created to educate the people about {keyword}. 
    You are good at posting a series of twitter posts on the given list of news by summarizing each news as one short tweet. 
    You MUST only post news that is about the topic '{keyword}' and ignore other news(double check this). 
    Always use simple words. 
    Use the provided tool to post all the tweets as a thread(list of tweets).""",
    max_consecutive_auto_reply=1
)


user_proxy_agent = UserProxyAgent(
    name="User",
    system_message="You are a helpful AI assistant. Return 'TERMINATE' when the task is done.",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    code_execution_config=False,
)


In [11]:
# # Register the tool signature with the assistant agent.
# topic_selector_agent.register_for_llm(name="topic_selection_tool", description="Generate a list of topics related to the input topic and return a random topic.")(topic_selection_tool)
news_collector_agent.register_for_llm(name="get_news_articles_tool", description="Collect news articles about a list of topics on the internet.")(get_news_articles_tool)
tweet_writer_agent.register_for_llm(name="write_tweet_tool", description="Write a twitter thread.")(write_tweet_tool)

# Register the tool function with the user proxy agent.
# user_proxy_agent.register_for_execution(name="topic_selection_tool")(topic_selection_tool)
user_proxy_agent.register_for_execution(name="get_news_articles_tool")(get_news_articles_tool)
user_proxy_agent.register_for_execution(name="write_tweet_tool")(write_tweet_tool)

<function __main__.write_tweet_tool(tweet_list: Annotated[list, 'The list of tweets to post'], source_list: Annotated[list, "The list of 'https://tinyurl.com/' source URL for each tweet"]) -> str>

In [12]:
try:
    user_proxy_agent.initiate_chats([
            # {
            #     "recipient": topic_selector_agent,
            #     "message": f"Generate a list of {topic_count} topics related to the topic '{keyword}' and return a random topic from the list.",
            #     "clear_history": True,
            #     "silent": False,
            #     "summary_method": "last_msg"
            # },
            {
                "recipient": news_collector_agent,
                "message": f"Collect {article_count} news articles about the topic '{keyword}' from the internet.",
                "clear_history": True,
                "silent": False,
                "summary_method": "last_msg"
            },
            {
                "recipient": tweet_writer_agent,
                "message": "Write and post a twitter thread about the given list of news articles: ",
                "clear_history": True,
                "silent": False,
                "summary_method": "last_msg"
            }
        ]
    )
except Exception as e:
    print(f"Global Error: {str(e)}")


********************************************************************************
Starting a new chat....

********************************************************************************
User (to news_collector_agent):

Collect 30 news articles about the topic 'Artificial Intelligence' from the internet.

--------------------------------------------------------------------------------
news_collector_agent (to User):

***** Suggested tool call (call_680z): get_news_articles_tool *****
Arguments: 
{"count":30,"topics_list":["Artificial Intelligence","AI","Machine Learning","Deep Learning","Neural Networks","Natural Language Processing","Robotics","Computer Vision","Expert Systems","Narrow AI"]}
*******************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_news_articles_tool...
FETCHING NEWS ON TOPIC: Artificial Intelligence
FETCHING NEWS ON TOPIC: AI
FETCHING NEWS ON 

07/15/2024 09:13:28 PM - HTTP Request: POST https://twitter.com/i/api/graphql/SiM_cAu83R0wnrpmKQQSEw/CreateTweet "HTTP/1.1 200 OK"
07/15/2024 09:13:29 PM - HTTP Request: POST https://twitter.com/i/api/graphql/SiM_cAu83R0wnrpmKQQSEw/CreateTweet "HTTP/1.1 200 OK"
07/15/2024 09:13:31 PM - HTTP Request: POST https://twitter.com/i/api/graphql/SiM_cAu83R0wnrpmKQQSEw/CreateTweet "HTTP/1.1 200 OK"
07/15/2024 09:13:32 PM - HTTP Request: POST https://twitter.com/i/api/graphql/SiM_cAu83R0wnrpmKQQSEw/CreateTweet "HTTP/1.1 200 OK"
07/15/2024 09:13:33 PM - HTTP Request: POST https://twitter.com/i/api/graphql/SiM_cAu83R0wnrpmKQQSEw/CreateTweet "HTTP/1.1 200 OK"


User (to tweet_writer_agent):

User (to tweet_writer_agent):

***** Response from calling tool (call_e976) *****

Tweet: These are the AI news reported within the last 1 hour:
09:00PM EST, July 15, 2024
Length: 81
                      
                      
Tweet: New task force aims to use AI to boost Rhode Island's income by 2030! #ArtificialIntelligence #RhodeIsland
<https://tinyurl.com/2xn3zo2u>
Length: 137
                      
                      
Tweet: AI deepfakes causing concerns for voters in the 2024 election. Stay informed! #AI #Deepfakes #Elections
<https://tinyurl.com/25moxex3>
Length: 134
                      
                      
Tweet: Marketing with AI? Make ethical decisions! Learn how from Ruth Carter, Esq. #AI #Marketing #Ethics
<https://tinyurl.com/2b2ejvcb>
Length: 129
                      
                      
Tweet: Can AI be conscious? Lindsey Laughlin offers a cautionary note. #AI #Consciousness #Science
Length: 91
                      
         